In [10]:
from collections import defaultdict
import torch
import numpy as np
import h5py
import math
import random
from random import choice
# device_gpu = torch.device('cuda')
device_cpu = torch.device('cpu')
row = 4007


#   初始化质心 随机初始化
def random_init(dataset, num_centers,row):
    #总epoch行数
    num_points = row
    #print(num_points)
    #单个子patch数
    #数据列数
    dimension = dataset.shape[1]
    #print(dataset.shape)
    used = torch.zeros(row, dtype=torch.long)
    #print("used",used.shape)
    centers = []
    for i in range(num_centers):
        while True:
            #随机选取
            cur_id = random.randint(0, num_points - 1)

            if used[cur_id] > 0:
                continue
            used[cur_id] = 1
            if i == 0:
                centers = dataset[cur_id].reshape(1,-1)
                #print(centers.shape)
            else:
                centers = np.append(centers,dataset[cur_id].reshape(1,-1),axis=0)
            break

    #print(centers.shape)
    return centers
def choice_controid(dataset,new_centers):
    #总epoch行数
    num_points = row
    #print(num_points)
    #单个子patch数
    #数据列数
    dimension = dataset.shape[1]
    #print(dataset.shape)
    used = torch.zeros(row, dtype=torch.long)
    #print("used",used.shape)
    centers = []
    for i in range(new_centers):
        while True:
            #随机选取
            cur_id = random.randint(0, num_points - 1)

            if used[cur_id] > 0:
                continue
            used[cur_id] = 1
            if i == 0:
                centers = dataset[cur_id].reshape(1,-1)
                #print(centers.shape)
            else:
                centers = np.append(centers,dataset[cur_id].reshape(1,-1),axis=0)
            break
    print("新选的质总数",centers.shape)
    #print(centers.shape)
    return centers

def distance(point,centersi):
    distance = np.sum(np.exp(-centersi**2)-2*np.cos(centersi-point))
    return distance
#  #为每个数据点计算最近的中心
def compute_codes(dataset, centers,row):
    num_points = row
    dimension = dataset.shape[1]
    num_centers = centers.shape[0]  #质点数量
    # 5e8 should vary depending on the free memory on the GPU
    # Ideally, automatically ;)
    #chunk_size = int(5e8 / num_centers)
    codes = np.zeros((num_points,1),dtype=int)  #标签

    for p,point in enumerate(dataset):

        shortest_index = 0
        for i in range(len(centers)):
            val = distance(point.reshape(1,-1), centers[i].reshape(1,-1))
            if i == 0:
                shortest = val

            else:
                if val < shortest:
                    shortest = val
                    shortest_index = i
        codes[p] = shortest_index
    return codes

def point_avg(points,row,column):

    dimensions = len(points)  #某种标签内点个数

    new_center = np.zeros((row,column),dtype=float)

    allpoint=[]
    for index in range(dimensions):
        if index == 0:
            print("算平均值得维数",points[index].shape)
            allpoint = points[index]
        else:
            allpoint += points[index]

        break
        # average of each dimension
    new_center=allpoint/(index+1)
    return new_center

# 计算新的中心作为数据点形成集群的手段
def update_centers(dataset, codes, num_centers,rfsize):
    num_points = dataset.shape[0]
    dimension = dataset.shape[1]

    new_means = defaultdict(list)
    for index in range(num_points):
        point = dataset[index]
        code = codes[index]
        new_means[code[0]].append(point.reshape(-1,rfsize*rfsize))   #array转化为list

        # print(code)
        #print(point.shape)
        #print(new_means[code[0]][index].shape)



    for index_c,points in enumerate(new_means):
        print("标签数量",index_c)
        if index_c == 0:
            centers = point_avg(new_means[points],num_centers,dimension).reshape(-1,rfsize*rfsize)
        else:
            centers = np.append(centers,point_avg(new_means[points],num_centers,dimension).reshape(-1,rfsize*rfsize),axis=0)
    #再随机选取足够的质点
    centers = np.append(centers,choice_controid(dataset,num_centers-index_c-1).reshape(-1,rfsize*rfsize),axis=0)
    return centers

#聚类  迭代
def cluster(dataset, num_centers,row,iterations,rfsize):
    print("初始化质心开始")
    centers = random_init(dataset, num_centers,row)  #初始化质心
    print("初始化质心结束")
    print("------------------")
    print("计算数据点最近中心开始")
    codes = compute_codes(dataset, centers,row) #为数据点计算最近的中心

    print("计算中心结束")
    print("----------------------")
    print("计算新的质点开始")
    centers = update_centers(dataset, codes, num_centers,rfsize) #数据集 标签 质点总数

    new_codes = compute_codes(dataset, centers.reshape(-1,rfsize*rfsize),row)
    print("计算新的质点结束---------")
    num_iterations =  1#迭代次数
    while num_iterations <= iterations:
        centers = update_centers(dataset, codes, num_centers,rfsize)
        new_codes = compute_codes(dataset, centers,row)
        print(num_iterations)
        if num_iterations%10 ==0:
            print("iterations:",num_iterations)
        if (codes== new_codes).all is True:  #判断数组是否相等
            print('Converged in %d iterations' % num_iterations)
            break
        codes = new_codes
        num_iterations += 1
    return centers, codes





channels = ['ECG', 'EMG', 'EOG-R', 'EOG-L', 'O2-A1', 'C4-A1', 'F4-A1']
h5file = ["./sleepdata/h5data/slp0001.h5",
          "./sleepdata/h5data/slp0002.h5",
          "./sleepdata/h5data/slp0003.h5",
          "./sleepdata/h5data/slp0004.h5",
          "./sleepdata/h5data/slp0005.h5"]
complexd_h5file = ["./sleepdata/h5data/complexd_slp0001.h5",
                       "./sleepdata/h5data/complexd_slp0002.h5",
                       "./sleepdata/h5data/complexd_slp0003.h5",
                       "./sleepdata/h5data/complexd_slp0004.h5",
                       "./sleepdata/h5data/complexd_slp0005.h5"]
controid ="./sleepdata/h5data/controid.h5"
row = 0   #行数
num_centers1 = 100  #质点
num_centers2 = 140
rfsize1 = 9   # 卷积大小
rfsize2 = 7   # 卷积大小
patchs = []
num_iterations1 = 80
num_iterations2 = 150

def getPatches(alldata_angle,patchsize,row,patchs,rfsize,num_iterations,num_centers):


    all_choices=range(patchsize-rfsize+1)
    for i in range(row):
        c_x = random.choice(all_choices)
        c_y = random.choice(all_choices)
        patchi_angle=alldata_angle[i].reshape(patchsize,patchsize)[c_x:c_x+rfsize,:][:,c_y:c_y+rfsize].reshape(1,-1)
        #patchi_sin=alldata_sin[i].reshape(patchsize,patchsize)[c_x:c_x+rfsize,:][:,c_y:c_y+rfsize]
        if i == 0:
            #patchs=[np.concatenate((patchi_cos),axis=1)]  #将30s的epoch放入list中
            patchs=patchi_angle
            print(patchs.shape,patchi_angle.shape)
        else:
            #patchs=np.append(patchs,[np.concatenate((patchi_cos,patchi_sin),axis=1)],axis=0) #将30s的epoch纵向拼接
            patchs=np.append(patchs,patchi_angle,axis=0) #将30s的epoch纵向拼接
            #print(patchs.shape)
    print(len(patchs),row)
    centers, codes = cluster(patchs, num_centers,row,num_iterations,rfsize)

    return centers, codes

# for filei in range(5):
#     fileh5 = h5py.File(complexd_h5file[filei], 'r')
#     row +=fileh5[channels[5]+"_angle"].value.shape[0]  #行数
#     column = 30*fileh5['sample_rate'][0] ##30秒epochs  列数
#     patchsize = int(math.sqrt(column))
#
#     if filei==0:
#         alldata_angle=fileh5[channels[5]+"_angle"].value
#         #alldata_sin=fileh5[channels[4]+"_sin"].value
#     else:
#         alldata_angle=np.vstack((alldata_angle,fileh5[channels[5]+"_angle"].value))
#         #alldata_sin=np.vstack((alldata_sin,fileh5[channels[4]+"_sin"].value))
#
#     fileh5.close()
#     # print("开始-----------")
#     # print(alldata_cos)
#     # print(alldata_sin)
#     # print("结束------------")
#
#     all_choices=range(patchsize-rfsize1+1)
# for i in range(row):
#     c_x = random.choice(all_choices)
#     c_y = random.choice(all_choices)
#     patchi_angle=alldata_angle[i].reshape(patchsize,patchsize)[c_x:c_x+rfsize1,:][:,c_y:c_y+rfsize1].reshape(1,-1)
#     #patchi_sin=alldata_sin[i].reshape(patchsize,patchsize)[c_x:c_x+rfsize,:][:,c_y:c_y+rfsize]
#     if i == 0:
#         #patchs=[np.concatenate((patchi_cos),axis=1)]  #将30s的epoch放入list中
#         patchs=patchi_angle
#         print(patchs.shape,patchi_angle.shape)
#     else:
#         #patchs=np.append(patchs,[np.concatenate((patchi_cos,patchi_sin),axis=1)],axis=0) #将30s的epoch纵向拼接
#         patchs=np.append(patchs,patchi_angle,axis=0) #将30s的epoch纵向拼接
#         #print(patchs.shape)


for filei in range(5):
        fileh5 = h5py.File(complexd_h5file[filei], 'r')

        row +=fileh5[channels[5]+"_angle"].value.shape[0]  #行数
        column = 30*fileh5['sample_rate'][0] ##30秒epochs  列数
        patchsize = int(math.sqrt(column))

        if filei==0:
            alldata_angle=fileh5[channels[5]+"_angle"].value
            #alldata_sin=fileh5[channels[4]+"_sin"].value
        else:
            alldata_angle=np.vstack((alldata_angle,fileh5[channels[5]+"_angle"].value))
            #alldata_sin=np.vstack((alldata_sin,fileh5[channels[4]+"_sin"].value))


        fileh5.close()
        # print("开始-----------")
        # print(alldata_cos)
        # print(alldata_sin)
        # print("结束------------")

print('Starting clustering')

centers1,codes1 = getPatches(alldata_angle,patchsize,row,patchs,rfsize1,num_iterations1,num_centers1)

centers2, codes2 = getPatches(alldata_angle,patchsize,row,patchs,rfsize2,num_iterations2,num_centers2)

print("End clustering")



filecon = h5py.File(controid, 'w')
filecon["controid1"] = centers1
filecon["controid2"] = centers2
filecon.close()
print("End")




D:\WorkSoftware\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:260: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
D:\WorkSoftware\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:265: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
D:\WorkSoftware\anaconda3\envs\pytorch\lib\site-packages\ipykernel_launcher.py:268: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.


Starting clustering
(1, 81) (1, 81)
4007 4007
初始化质心开始
初始化质心结束
------------------
计算数据点最近中心开始
计算中心结束
----------------------
计算新的质点开始
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
标签数量 2
算平均值得维数 (1, 81)
标签数量 3
算平均值得维数 (1, 81)
标签数量 4
算平均值得维数 (1, 81)
新选的质总数 (95, 81)
计算新的质点结束---------
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
标签数量 2
算平均值得维数 (1, 81)
标签数量 3
算平均值得维数 (1, 81)
标签数量 4
算平均值得维数 (1, 81)
新选的质总数 (95, 81)
1
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
标签数量 2
算平均值得维数 (1, 81)
标签数量 3
算平均值得维数 (1, 81)
新选的质总数 (96, 81)
2
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
新选的质总数 (98, 81)
3
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
新选的质总数 (98, 81)
4
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
标签数量 2
算平均值得维数 (1, 81)
新选的质总数 (97, 81)
5
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
标签数量 2
算平均值得维数 (1, 81)
标签数量 3
算平均值得维数 (1, 81)
标签数量 4
算平均值得维数 (1, 81)
标签数量 5
算平均值得维数 (1, 81)
标签数量 6
算平均值得维数 (1, 81)
新选的质总数 (93, 81)
6
标签数量 0
算平均值得维数 (1, 81)
标签数量 1
算平均值得维数 (1, 81)
新选的质总数 (98, 81)
7
标签数量 0
算平均值得维数 (1,